In [1]:
from kerax.layers import Conv2D, Dense, Flatten, Input, BatchNormalization, Activation, Add, MaxPool2D, Weight
from kerax.models import Model, Sequential
from kerax.losses import CategoricalCrossEntropy
import numpy as np
from kerax import activations
from kerax.utils import to_categorical, to_numbers
from jax.random import PRNGKey
import pandas as pd
import matplotlib.pyplot as plt
from jax import vmap
from jax import random
from jax import lax
from jax import numpy as jnp
import optax as opx
from jax import value_and_grad, grad
from kerax import optimizers

In [4]:
inputs = Input((1, 784))
dense = Dense()

In [ ]:
inputs = Input((28, 28, 1))
conv1 = Conv2D(64, 3, key=PRNGKey(100))(inputs)
act1 = Activation('relu')(conv1)
conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(act1)
act2 = Activation('relu')(conv3)
maxpool = MaxPool2D((2,2))(act2)
conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(105))(maxpool)
flatten = Flatten()(conv4)
dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(10)(dense1)


AssertionError: length mismatch: [4, 2, 2]

In [3]:
model = Model(input=inputs, output=dense2)

In [4]:
loss = CategoricalCrossEntropy(model)

In [5]:
loss.name

'CategoricalCrossEntropy'

In [32]:
train_x = random.normal(PRNGKey(8), (128, 28, 28, 1))
label =random.randint(PRNGKey(3), (128, 1), 0, 2)

In [39]:
def loss(params, train_x, train_y):
    out = model.call_with_external_weights(params, train_x)
    out = opx.softmax_cross_entropy(out, train_y)
    out = jnp.mean(out)
    return out

In [98]:
loss_value, grads = value_and_grad(loss)(model.params, train_x, label)

In [84]:
optimizer = opx.adam(0.001)

In [85]:
state = optimizer.init(model.params)

In [99]:
new_params, state = optimizer.update(grads, state)

In [100]:
new_params = opx.apply_updates(model.params, new_params)

In [101]:
model.update_weights(new_params)

In [102]:
loss_value

DeviceArray(12.063399, dtype=float32)